In [ ]:
#@title **1 - INITIATION**
import os, multiprocessing, subprocess, contextlib, time, threading
from google.colab import output

USERNAME = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
#@markdown *- Put your Mega account here -*

def login():
    os.environ['usrC'] = USERNAME
    os.environ['pwdC'] = PASSWORD
    !mega-login $usrC $pwdC

print('----- Installing Qbittorrent -----')
if not os.path.isfile('/usr/bin/qbittorrent-nox'):
    !apt install qbittorrent-nox
output.clear()
print("----- Installing MEGA -----")
if not os.path.exists("/usr/bin/mega-cmd"):
    !apt-get -y update')
    !apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
    !curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb
    !dpkg -i /var/cache/apt/archives/MEGAcmd.deb
    output.clear()
    print("MEGA is installed.")
print('----- Login in -----')
p = multiprocessing.Process(target=login)
p.start()
p.join(10)
if p.is_alive():
    p.terminate()
    p.join()
print('----- Completed -----')
!mega-whoami
print('----- Done -----')

In [ ]:
#@title **2. START QBITTORENT**
SAVE_PATH = '/content/download'

if not os.path.isdir(SAVE_PATH):
    os.mkdir(SAVE_PATH)
!mkdir -p -m 666 /{content/qBittorrent,root/{.qBittorrent_temp,.config/qBittorrent}} && curl -s https://pastebin.com/raw/zn6HBgHz -o /root/.config/qBittorrent/qBittorrent.conf
with open('/root/.config/qBittorrent/qBittorrent.conf', 'r') as f:
    content = f.read().replace('%SavePath%', SAVE_PATH)
with open('/root/.config/qBittorrent/qBittorrent.conf', 'w') as f:
    f.write(content)
!qbittorrent-nox -d --webui-port=4444
print('----- Open this link to open QBittorrent -----')
print(output.eval_js("google.colab.kernel.proxyPort(4444)"))

In [ ]:
#@title **2.1 - INFINITY RUN**
#@markdown *to keep Colab from auto-terminating*
#@markdown <br>*- Thank to John Titor code -*
while True:pass


In [ ]:
#@title **3. UPLOADING TO MEGA**
#@markdown *only run this cell after you're done torrenting*
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def clear_output():
    while True:
        time.sleep(60)
        output.clear()
        print('----- Transfering -----')

threading.Thread(target=clear_output).start()
print('----- Transfering -----')
file_list = [os.path.join(SAVE_PATH, f) for f in os.listdir(SAVE_PATH)]
for f in file_list:
    print(f'----- Uploading {f} -----')
    cmd = ['mega-put', '-c', '--ignore-quota-warn', f]
    proc = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
        )
    for line in unbuffered(proc):
        print(line)
print('----- Completed -----')
